<a href="https://colab.research.google.com/github/imstaHub/hanghae99/blob/master/week3_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week3 Basic Homework

In [1]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [2]:
import torch

#device setting
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.backends.cuda.is_built():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cuda


In [21]:
def accuracy(model, dataloader, device):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)
    #preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

## [MY CODE] tokenizer, dataset, model 로드
- tokenizer: distilbert
- dataset: fancyzhx/ag_news
- model: distilbert

In [4]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hu

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
ds = load_dataset("fancyzhx/ag_news")


def collate_fn(batch):
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  # truncation 옵션 제거
  # -> 학습 중 warning과 학습이 잘 되지 않아서 positional encoding layer의 input에 맞춤. 다행히 토큰 최대 길이가 512를 넘지 않았음
  texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=512).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)

In [13]:
# load model and check params
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): L

## [MY CODE] dataset 확인

In [14]:
# news 분류
ds['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [15]:
# news 내용
ds['train']['text'][:10]

["Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.',
 "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.",
 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.',
 'Oil prices soar to all-time record, posing new menace to US economy (A

In [16]:
# news 유형
for i in list(dict.fromkeys(ds['train']['label'])):
    cnt =ds['train']['label'].count(i)
    print(f'{i} : {cnt}')

2 : 30000
3 : 30000
1 : 30000
0 : 30000


In [37]:
# 최대 토큰 길이 확인
tmp_list = []
for data in train_loader:
  inputs, labels = data
  for i in range(len(inputs)):
    tmp_list.append(list(inputs[i]!=0).count(True))

print(max(tmp_list))

379


## [MY CODE] model 정의, 다중분류

In [17]:
from torch import nn
from torch.optim import Adam
import numpy as np

class TextClassifier(nn.Module):
  def __init__(self, output_dims):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.classifier = nn.Linear(768, output_dims) # 마지막에 뉴스 유형의 수만큼 output으로 return

  def forward(self, x):
    x = self.encoder(x)['last_hidden_state']
    x = self.classifier(x[:, 0])

    return x

output_dims = len(list(dict.fromkeys(ds['train']['label'])))
model = TextClassifier(output_dims=output_dims)

# encoder(last hidden state까지) freeze
for param in model.encoder.parameters():
  param.requires_grad = False


model = model.to(device)
# loss function 변경, 다중분류
loss_fn = nn.CrossEntropyLoss()

lr = 0.001
optimizer = Adam(model.parameters(), lr=lr)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


## [MY CODE] 학습 진행

In [18]:
import time

n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()

  start = time.time()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device).long()

    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  end = time.time() - start
  print(f"Epoch {epoch:3d} | Time : {end} | Train Loss: {total_loss}")


Epoch   0 | Time : 472.1220202445984 | Train Loss: 884.0773583799601
Epoch   1 | Time : 471.232390165329 | Train Loss: 697.9789187684655
Epoch   2 | Time : 471.40498065948486 | Train Loss: 674.1229237094522
Epoch   3 | Time : 470.85683941841125 | Train Loss: 658.2343865558505
Epoch   4 | Time : 471.45809841156006 | Train Loss: 652.4805552661419
Epoch   5 | Time : 473.0971884727478 | Train Loss: 646.8016795143485
Epoch   6 | Time : 471.1451210975647 | Train Loss: 641.7652831673622
Epoch   7 | Time : 470.8661398887634 | Train Loss: 641.5526793636382
Epoch   8 | Time : 469.5847589969635 | Train Loss: 637.9806367903948
Epoch   9 | Time : 471.71616554260254 | Train Loss: 636.1580523177981


In [22]:
with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader, device)
  test_acc = accuracy(model, test_loader, device)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.891 | Test acc: 0.889


## [LOG] train, test 둘다 정확도 89%로, 학습이 잘 되었다.